In [36]:
import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from scipy.interpolate import interp1d

In [2]:
%config InlineBackend.figure_format = "retina"

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 18

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.9 * fs,
          'ytick.labelsize': 0.9 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)

# Part a

In [3]:
def fermi_dirac(j, z):
    assert j in [0.5, 1.5]
    
    if j == 0.5:
        a = 9.6
        b = 2.13
        c = 12 / 5
    else:
        a = 14.9
        b = 2.64
        c = 9 / 4
        
    first_term = (j + 1) * 2**(j + 1) / (b + z + (np.abs(z - b)**c + a)**(1/c))**(j + 1)
    second_term = np.exp(-z) / gamma(j + 1)
    
    return (first_term + second_term)**(-1)

In [39]:
rho_c = 325 * u.g / u.cm**3
T_c = 6e6 * u.K
s = 0.5

n = (1.2 * rho_c / (1.4 * const.m_p)).to(u.cm**(-3))
lam = (const.h / (2 * np.pi * const.m_e * const.k_B * T_c)**(1/2)).to(u.cm)

fd = n * lam**3 * np.pi**(1/2) / (2 * (2 * s + 1))

print(fd)

z_range = np.linspace(-10, 50, 1000)
inverse_fd_func = interp1d(fermi_dirac(0.5, z_range), z_range)

fugacity = inverse_fd_func(fd)
print(fugacity)

2.079537345861597
1.650064774393406


# Part b

In [42]:
pressure = 4 * (2 * s + 1) / (3 * np.pi**(1/2) * lam**3) * fermi_dirac(1.5, fugacity) * const.k_B * T_c

In [50]:
pressure.to(u.Pa)

<Quantity 1.92124563e+16 Pa>

In [48]:
pressure.to(u.g * u.cm**(-1) * u.s**(-2))

<Quantity 1.92124563e+17 g / (cm s2)>

# Part d

In [51]:
ideal_pressure = n * const.k_B * T_c

In [53]:
ideal_pressure.to(u.Pa)

<Quantity 1.37966396e+16 Pa>

# Part e

In [84]:
mu_e = 1.1
first_term = 0.13 / (mu_e * fermi_dirac(0.5, fugacity))**(2/3)
second_term = rho_c / (8.44 * u.g / u.cm**3)

In [85]:
radius = (first_term * second_term**(-1/3))**(1/2) * u.R_sun
mass = rho_c * radius.value**3 / (8.44 * u.g / u.cm**3) * u.Msun

In [86]:
radius

<Quantity 0.14891199 solRad>

In [87]:
mass

<Quantity 0.12715396 solMass>